In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
html_page = requests.get('https://www.imdb.com/search/title/?title_type=feature&release_date=2010-01-01,2019-12-31')

In [4]:
html_page

<Response [200]>

In [5]:
soup = BeautifulSoup(html_page.content, 'html.parser')

In [6]:
type(soup)

bs4.BeautifulSoup

In [7]:
movies_container = soup.find('div', class_='lister-list')

In [8]:
titles = movies_container.findAll('h3')
titles[0]

<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt6751668/">Parasite</a>
<span class="lister-item-year text-muted unbold">(2019)</span>
</h3>

In [9]:
titles[0].find('a')

<a href="/title/tt6751668/">Parasite</a>

In [10]:
titles[0].find('a').get_text()

'Parasite'

In [11]:
imdb_movie_list = movies_container.findAll('div', class_="lister-item-content")

In [12]:
def titles(movie_container):
    all_titles = [h3.find('a').get_text() for h3 in movies_container.findAll('h3')]
    return all_titles

In [13]:
def mpaa_ratings(imdb_movie_list):
    mpaa_ratings = []
    for movie in imdb_movie_list:
        rating = movie.find('span', class_='certificate')
        if rating:
            mpaa_ratings.append(rating.get_text())
        else:
            mpaa_ratings.append("NaN")
    return mpaa_ratings

In [14]:
def runtimes(imdb_movie_list):
    all_runtimes = []
    for movie in imdb_movie_list:
        runtime = movie.find('span', class_='runtime')
        if runtime:
            all_runtimes.append(runtime.get_text())
        else:
            all_runtimes.append('NaN')
    return all_runtimes

In [15]:
def genres(imdb_movie_list):
    all_genres = []
    for movie in imdb_movie_list:
        genre = movie.find('span', class_='genre')
        if genre:
            all_genres.append(genre.get_text())
        else:
            all_genres.append("NaN")
    all_genres = [genre.strip() for genre in all_genres]
    return all_genres

In [16]:
def imdb_id(imdb_movie_list):
    all_ids = []
    for movie in imdb_movie_list:
        header = movie.find('h3', class_="lister-item-header")
        xid = header.find('a').attrs['href']
        xid = xid.lstrip('/title/')
        xid = xid.rstrip('/')
        xid = "tt"+xid
        all_ids.append(xid)
    return all_ids

In [17]:
def star_ratings(imdb_movie_list):
    star_ratings = []
    for movie in imdb_movie_list:
        rating = movie.find('strong')
        if rating:
            star_ratings.append(rating.get_text())
        else:
            star_ratings.append("NaN")
    return star_ratings

In [18]:
import time

In [19]:
#collect data from all pages
all_titles = []
all_mpaa_ratings = []
all_runtimes = []
all_genres = []
all_star_ratings = []
all_ids = []

#first page
html_page = requests.get("https://www.imdb.com/search/title/?title_type=feature&release_date=2010-01-01,2019-12-31")
soup = BeautifulSoup(html_page.content, 'html.parser')
movies_container = soup.find('div', class_="lister-list")
imdb_movie_list = movies_container.findAll('div', class_="lister-item-content")
for title in titles(movies_container):
    all_titles.append(title)
for rating in mpaa_ratings(imdb_movie_list):
    all_mpaa_ratings.append(rating)
for runtime in runtimes(imdb_movie_list):
    all_runtimes.append(runtime)
for genre in genres(imdb_movie_list):
    all_genres.append(genre)
for rating in star_ratings(imdb_movie_list):
    all_star_ratings.append(rating)
for xid in imdb_id(imdb_movie_list):
    all_ids.append(xid)

#for every other page
for i in range(51,10_001,50):
    url = f"https://www.imdb.com/search/title/?title_type=feature&release_date=2010-01-01,2019-12-31&start={i}&ref_=adv_nxt"
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    movies_container = soup.find('div', class_="lister-list")
    imdb_movie_list = movies_container.findAll('div', class_="lister-item-content")
    for title in titles(movies_container):
        all_titles.append(title)
    for rating in mpaa_ratings(imdb_movie_list):
        all_mpaa_ratings.append(rating)
    for runtime in runtimes(imdb_movie_list):
        all_runtimes.append(runtime)
    for genre in genres(imdb_movie_list):
        all_genres.append(genre)
    for rating in star_ratings(imdb_movie_list):
        all_star_ratings.append(rating)
    for xid in imdb_id(imdb_movie_list):
        all_ids.append(xid)
    time.sleep(0.5)

In [21]:
movie_dict = {'movie_id': all_ids, 'movie_title': all_titles, 'mpaa_rating': all_mpaa_ratings, 'runtime': all_runtimes, 'genre': all_genres, 'star_rating': all_star_ratings}

In [22]:
import pandas as pd

In [23]:
df = pd.DataFrame(movie_dict)
df.head()

movie_id    movie_title mpaa_rating  runtime                    genre  \
0  tt6751668       Parasite           R  132 min  Comedy, Drama, Thriller   
1  tt8946378     Knives Out       PG-13  131 min     Comedy, Crime, Drama   
2  tt2584384    Jojo Rabbit       PG-13  108 min       Comedy, Drama, War   
3  tt8579674           1917           R  119 min               Drama, War   
4  tt8367814  The Gentlemen           R  113 min    Action, Comedy, Crime   

  star_rating  
0         8.6  
1         8.0  
2         8.0  
3         8.4  
4         8.1

In [24]:
df.to_csv('IMDB_Data')